In [2]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials as yfs
import json

In [1]:
pip install yahoofinancials


  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15195 sha256=ea150ca34599abf2043a933a1b953379bd866b7604689223e628dd42530136cd
  Stored in directory: c:\users\tafim\appdata\local\pip\cache\wheels\6a\90\0c\08c7ac2ce60b9ac91529417d471e59244f9f96848c86f14809
Successfully built yahoofinancials


In [12]:
def Sheet_Data(ticker,sheet_data):
    save = []
    
    df = pd.DataFrame.from_dict({(i,j): sheet_data[i][j] 
                            for i in sheet_data.keys() 
                            for j in sheet_data[i].keys()},
                        orient='index')

    for i in df.columns:
        df[i].tolist()
        df2 = pd.json_normalize(df[i],max_level = 1)

        #Gets  Date and renames 
        Date = df2.columns[0]
        Date = Date[0:11]
        df2.index = df2.index.rename(ticker)
        df2.columns = df2.columns.str.replace(str(Date),'')

        #Tranposes DataFrame and changes it to the 
        df2 = df2.T
        df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:10]))},inplace = True)
        #display(df2)
        save.append(df2)
    df3 = pd.concat(save,axis = 1)
    display(df3)
    filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
    print(filename)
    #df3.to_csv('{}.csv'.format(filename))

#filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
#df3.t

In [13]:
ticker = 'AAPL'
yahoo_financials = yfs(ticker)

cash = yahoo_financials.get_financial_stmts('annual','cash')
income = yahoo_financials.get_financial_stmts('annual','income')
balance = yahoo_financials.get_financial_stmts('annual','balance')

In [14]:
Sheet_Data(ticker,cash)
Sheet_Data(ticker,income)
Sheet_Data(ticker,balance)

AAPL,2020-09-26,2019-09-28,2018-09-29,2017-09-30
investments,5335000000,58093000000,30845000000,-33542000000
changeToLiabilities,-1981000000,-2548000000,9172000000,8373000000
totalCashflowsFromInvestingActivities,-4289000000,45896000000,16066000000,-46446000000
netBorrowings,2499000000,-7819000000,432000000,29014000000
totalCashFromFinancingActivities,-86820000000,-90976000000,-87876000000,-17974000000
changeToOperatingActivities,881000000,-896000000,30016000000,-8480000000
issuanceOfStock,880000000,781000000,669000000,555000000
netIncome,57411000000,55256000000,59531000000,48351000000
changeInCash,-10435000000,24311000000,5624000000,-195000000
repurchaseOfStock,-75992000000,-69714000000,-75265000000,-34774000000


AAPL_cashflowStatementHistory


AAPL,2020-09-26,2019-09-28,2018-09-29,2017-09-30
researchDevelopment,18752000000,16217000000,14236000000,11581000000
effectOfAccountingCharges,None,None,None,None
incomeBeforeTax,67091000000,65737000000,72903000000,64089000000
minorityInterest,None,None,None,None
netIncome,57411000000,55256000000,59531000000,48351000000
sellingGeneralAdministrative,19916000000,18245000000,16705000000,15261000000
grossProfit,104956000000,98392000000,101839000000,88186000000
ebit,66288000000,63930000000,70898000000,61344000000
operatingIncome,66288000000,63930000000,70898000000,61344000000
otherOperatingExpenses,None,None,None,None


AAPL_incomeStatementHistory


AAPL,2020-09-26,2019-09-28,2018-09-29,2017-09-30
totalLiab,258549000000,248028000000,258578000000,241272000000
totalStockholderEquity,65339000000,90488000000,107147000000,134047000000
otherCurrentLiab,47867000000,43242000000,39293000000,38099000000
totalAssets,323888000000,338516000000,365725000000,375319000000
commonStock,50779000000,45174000000,40201000000,35867000000
otherCurrentAssets,11264000000,12352000000,12087000000,13936000000
retainedEarnings,14966000000,45898000000,70400000000,98330000000
otherLiab,46108000000,50503000000,48914000000,43251000000
treasuryStock,-406000000,-584000000,-3454000000,-150000000
otherAssets,33952000000,32978000000,22283000000,18177000000


AAPL_balanceSheetHistory


# Testing for working all of the files work

df = pd.DataFrame.from_dict({(i,j): balance_sheet_data[i][j] 
                           for i in balance_sheet_data.keys() 
                           for j in balance_sheet_data[i].keys()},
                       orient='index')
for i in df.columns:
    df[i].tolist()
    df2 = pd.json_normalize(df[i],max_level = 1)

    #Gets  Date and renames 
    Date = df2.columns[0]
    Date = Date[0:11]
    df2.index = df2.index.rename(ticker)
    df2.columns = df2.columns.str.replace(str(Date),'')

    #Tranposes DataFrame and changes it to the 
    df2 = df2.T
    df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:10]))},inplace = True)
    #display(df2)
    save.append(df2)
df3 = pd.concat(save,axis = 1)
display(df3)